# Classes - Itermediate Concepts

## Magic/Dunder methods

Under the hood basically all Python data structures are objects:

In [ ]:
print(isinstance(int, object))
print(isinstance(str, object))
print(isinstance(dict, object))
print(isinstance(list, object))

We can see a list of all their methods using the dir() function

In [ ]:
dir(list)

We can see some of our favourite methods such as append here. But theres also a lot of methods beginning and ending with __ . These are special methods for objects in Python that help us implement useful functionality. 

### __repr__ method

When it comes to printing out dictionaries we get a really nice visual representation of them

In [ ]:
x = {"key":"value"}
print(x)

If we implement a very similar structure as class we don't get such a nice output from the print statement

In [ ]:
class KVPair:
    def __init__(self, value: str) -> None:
        self.key = value

x = KVPair("value")
print(x)

If we choose to implement (add) a function called __repr__ we can tell Python how the print function should act on this class

In [ ]:
class KVPair:
    def __init__(self, value: str) -> None:
        self.key = value
    
    def __repr__(self) -> str:
        return f"key: {self.key}"

x = KVPair("value")
print(x)

### __eq__ method

It is often useful in Python to compare two data types using the == operator to see if they are semantically the same

In [ ]:
x = {"key":"value"}
y = {"key":"value"}
print(x==y)

This however does not work out of the box with objects:

In [ ]:
x = KVPair("value")
y = KVPair("value")
print(x==y)

We can "fix" this with the __eq__ method

In [ ]:
class KVPair:
    def __init__(self, value: str) -> None:
        self.key = value
    
    def __repr__(self) -> str:
        return f"key: {self.key}"

    def __eq__(self, other: object) -> bool:
        if type(self) == type(other):
            return self.key == other.key
        return False

In [ ]:
x = KVPair("value")
y = KVPair("value")
print(x==y)

### Ordering methods

Sometimes it is useful to be able to apply ordering to objects. That is to say an object is bigger than another

In [ ]:
x = 3
y = 7
y > x

By default objects can't be compared"

In [ ]:
x = KVPair("value")
y = KVPair("value")
print(x>y)

We can however use the __gt__ greater than, __lt__ less than, __le__ less than or equal to, __ge__ greater than or equal to methods to add this functionality

In [ ]:
class KVPair:
    def __init__(self, value: str) -> None:
        self.key = value
    
    def __repr__(self) -> str:
        return f"key: {self.key}"

    def __eq__(self, other: object) -> bool:
        if type(self) == type(other):
            return self.key == other.key
        return False
    
    def __gt__(self, other: "KVPair") -> bool:
        if type(self) == type(other): 
            return self.key > other.key
        raise TypeError(f"Cannot compare type {type(other)} with {type(self)}")

In [ ]:
x = KVPair("apple")
y = KVPair("banana")
print(x>y)

If we have implemented this method we can actually now sort list of our objects using the sorted function:

In [ ]:
obj_list = [y, x]
print(obj_list)
sorted_obj_list = sorted(obj_list)
print(sorted_obj_list)

## Dunder Attributes

There are also special attributes that begin with __ in python. 

Two useful ones are __doc__ which displays the class's docstring and __dict__ which returns the objects attributes as a dictionary

In [ ]:
class KVPair:
    """KVPair's docstring"""
    def __init__(self, value: str) -> None:
        self.key = value

In [ ]:
x = KVPair("apple")
print(x.__doc__)
print(x.__dict__)

## Python's Data Model

There are many more magic methods that can be implemented. The best place to find out more is to read the Python Data Model documentation: https://docs.python.org/3/reference/datamodel.html

# Class and Static Methods

## Binding concepts to classes

We can actually bind data to classes rather than objects

In [ ]:
class Circle:
    pi = 3.14159
    
    def __init__(self, radius: float):
        self.radius = radius
        
    def compute_circumference(self):
        return self.pi*self.radius*2

We can access the value of pi as follows

In [ ]:
Circle.pi

In [ ]:
circle = Circle(2)
circle.compute_circumference()

In principle you can actaully change the class attribute value at run time (DONT it will make your code VERY confusing)

In [ ]:
circle_old = Circle(2)
Circle.pi = 3
circle_new = Circle(2)
print(circle_old.compute_circumference())
print(circle_new.compute_circumference())

But the point is we can bind concepts to the class and access our class in the code as if it was an object. We can use this idea to help group concepts together.

## Class methods

The idea of class methods is that rather than having access to the self property (ie the object being constructed) they have access to the underlying class. We can use this generate classes in alternative ways to the __init__ method

In [ ]:
class Circle:
    pi = 3.14159
    
    def __init__(self, radius: float):
        self.radius = radius
    
    def __repr__(self) -> float:
        return f"A circle with radius: {self.radius}"
        
    def compute_circumference(self):
        return self.pi*self.radius*2
    
    @classmethod
    def from_circumference(cls, circumference: float) -> "Circle":
        radius = circumference/2/cls.pi
        return cls(radius)

In [ ]:
circle = Circle(2.0)
print(circle)
circle = Circle.from_circumference(12.56636)
print(circle)

To make a method into a class method we apply the @classmethod decorator(https://www.geeksforgeeks.org/decorators-in-python/) to it (this essientally alters the behaviour. Then similar to the init method the first parameter is special but rather than providing access to the underlying object it provides access to the underlying class. By convention we call it cls.

It is worth contrasting this functionality against an alternative implementation

In [ ]:
class Circle:
    pi = 3.14159
    
    def __init__(self, radius: float = None, circumference = None):
        if radius and not circumference:
            self.radius = radius
        elif circumference and not radius:
            self.radius = circumference/2/self.pi
        else:
            raise ValueError("Provide either radius or circumference exclusively")
    
    def __repr__(self) -> float:
        return f"A circle with radius: {self.radius}"

circle = Circle(radius=2.0)
print(circle)
circle = Circle(circumference=12.56636)
print(circle)

The __init__ method is now pretty clunky and has low cohesion. 
Imagine you now need to be able to construct circle knowing only their area! This creates complex logic in the __init__ method and requires the constructor to be rewritten (who knows what unexpected changes this has upstream!). It's trivial to extend this with class methods

In [ ]:
import math

class Circle:
    pi = 3.14159
    
    def __init__(self, radius: float):
        self.radius = radius
    
    def __repr__(self) -> float:
        return f"A circle with radius: {self.radius}"
        
    def compute_circumference(self):
        return self.pi*self.radius*2
    
    @classmethod
    def from_circumference(cls, circumference: float) -> "Circle":
        radius = circumference/2/cls.pi
        return cls(radius)
    
    @classmethod
    def from_area(cls, area: float) -> "Circle":
        radius = math.sqrt(2*area/cls.pi)
        return cls(radius)

In [ ]:
circle = Circle.from_area(4)
print(circle)

## Static Methods

Sometimes we want to add a method to a class that doesn't require access to the underlying object. This is somewhat unecessary in Python from a functionality perspective but it can be useful from a code organisation perspective. That is this method belongs with this class and is probably not used elsewhere

The below example is not something you'd really do in Python (you'd use a module to implement this functionality) but it's useful to see it in action

In [ ]:
class Mathematics:
    
    @staticmethod
    def add(x: float, y: float) -> float:
        return x + y

print(Mathematics.add(1,2))

Personally I use them to tidy up standard methods in classes. Returning to the circle example:

In [ ]:
import math

class Circle:
    pi = 3.14159
    
    def __init__(self, radius: float):
        self.radius = radius
    
    def __repr__(self) -> float:
        return f"A circle with radius: {self.radius}"
        
    def compute_circumference(self):
        return self.pi*self.radius*2
    
    @staticmethod
    def compute_radius_from_area(area: float) -> float:
        return math.sqrt(2*area/Circle.pi)
    
    @staticmethod
    def compute_radius_from_circumference(circumference: float) -> float:
        return circumference/2/Circle.pi
    
    @classmethod
    def from_circumference(cls, circumference: float) -> "Circle":
        radius = cls.compute_radius_from_circumference(circumference)
        return cls(radius)
    
    @classmethod
    def from_area(cls, area: float) -> "Circle":
        radius = cls.compute_radius_from_area(area)
        return cls(radius)

circle = Circle.from_area(4)
print(circle)

circle = Circle.from_circumference(12.56636)
print(circle)

# Inheritance

Much like in the real world some categories are sub categories of others. For example we could have a class that represented a Person

In [ ]:
class Person:
    
    def __init__(self, name: str, age: int, haircut: str):
        self.name = name
        self.age = age
        self.haircut = haircut
    
    def get_haircut(self):
        self.haircut = "short"
        
    def grow_hair(self):
        self.haircut = "long"    

Now some people could be Employees and we'd want a class to represent them. They're also going to be People in our system so they will need all the functionality of people. We might implement them as such

In [ ]:
class Employee:
    
    def __init__(self, name: str, age: int, haircut: str, role: str):
        self.name = name
        self.age = age
        self.haircut = haircut
        self.role = role
    
    def get_haircut(self):
        self.haircut = "short"
        
    def grow_hair(self):
        self.haircut = "long"    
    
    def change_role(self, new_role: str):
        self.role = new_role

While this works it has a key draw back that the relationship between employee and person isn't really established in the code. If we changed Person this wouldn't be reflected in Employee we'd have to remember to update employee. This is the same problem we always have when we copy and paste code. 

We can fix this issue with a concept known as inhertitance. This is written as follows:

In [ ]:
class Employee(Person):
    
    def __init__(self, name: str, age: int, haircut: str, role: str):
        super().__init__(name, age, haircut)
        self.role = role 
    
    def change_role(self, new_role: str):
        self.role = new_role

Let's see that the class functions as expected:

In [ ]:
tim = Employee("Tim", 34 , "long", "Pilot")
print(tim.__dict__)
tim.get_haircut()
print(tim.__dict__)

In [ ]:
sarah = Person("Sarah", 43, "long")
print(sarah.__dict__)

In [ ]:
print(isinstance(tim, Employee))
print(isinstance(tim, Person))
print(isinstance(sarah, Employee))
print(isinstance(sarah, Person))

The super() function in the above example is used to access the superclasses constructor.

We can actually inherit from an inherited class 

In [ ]:
class PartTimeEmployee(Employee):
     def __init__(self, name: str, age: int, haircut: str, role: str, weekly_hours: float):
        super().__init__(name, age, haircut, role)
        self.weekly_hours = weekly_hours 

sam = PartTimeEmployee("Sam", 23, "short", "Butcher", 20)
print(sam.__dict__)
print(isinstance(sam, PartTimeEmployee))
print(isinstance(sam, Employee))
print(isinstance(sam, Person))

Be careful with this! You can create extremely complex and highly coupled code if you have too many layers of inheritence. If you feel you need more than 2 layera you should probably rethink your design.